In [1]:
pip install opencv-python-headless numpy open3d


^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import cv2
import numpy as np

# Path to the folder containing images
image_folder_path = "C:/Users/Hope.KingsMan/Downloads/Documents/100831_155323_MultiCamera0_subset_db"

# Stereo matching parameters
stereo = cv2.StereoBM_create(numDisparities=64, blockSize=15)

# Intrinsic parameters (example values, replace with dataset-specific values)
focal_length = 718.856
baseline = 0.573

# Function to compute depth map
def compute_depth_map(left_img, right_img):
    if left_img is None or right_img is None:
        raise ValueError("One of the images is empty. Check the file paths.")
    
    # Convert to grayscale
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
    
    # Compute disparity
    disparity = stereo.compute(left_gray, right_gray).astype(np.float32) / 16.0
    
    # Avoid division by zero
    disparity[disparity <= 0] = 0.1
    
    # Calculate depth
    depth = (focal_length * baseline) / disparity
    return depth

# Get sorted list of all files in the folder
files = sorted(os.listdir(image_folder_path))

# Process pairs of images
depth_maps = []
for i in range(0, len(files), 2):  # Assuming left and right images alternate
    left_img_path = os.path.join(image_folder_path, files[i])
    right_img_path = os.path.join(image_folder_path, files[i + 1])
    
    # Load images
    left_img = cv2.imread(left_img_path)
    right_img = cv2.imread(right_img_path)
    
    # Compute depth map
    try:
        depth = compute_depth_map(left_img, right_img)
        depth_maps.append(depth)
    except ValueError as e:
        print(f"Skipping pair {files[i]} and {files[i + 1]}: {e}")

# Output the number of depth maps generated
print(f"Generated depth maps for {len(depth_maps)} image pairs.")


import matplotlib.pyplot as plt

# Visualize the depth maps
for i, depth in enumerate(depth_maps):
    plt.figure(figsize=(10, 6))
    plt.imshow(depth, cmap='jet')  # Use 'jet' colormap for better visualization
    plt.colorbar(label="Depth (meters)")
    plt.title(f"Depth Map {i + 1}")
    plt.xlabel("Image Width (pixels)")
    plt.ylabel("Image Height (pixels)")
    plt.show()


Generated depth maps for 101 image pairs.


In [4]:
import open3d as o3d
import numpy as np

# Parameters (replace with actual values for your camera)
cx = 607.1928  # Principal point x
cy = 185.2157  # Principal point y
fx = 718.856   # Focal length x
fy = 718.856   # Focal length y

# Function to convert depth map to point cloud
def depth_to_point_cloud(depth_map):
    h, w = depth_map.shape
    points = []
    for v in range(h):
        for u in range(w):
            z = depth_map[v, u]
            if z > 0:
                x = (u - cx) * z / fx
                y = (v - cy) * z / fy
                points.append([x, y, z])
    return np.array(points)

# Convert depth maps to point clouds
point_clouds = []
for depth in depth_maps:
    points = depth_to_point_cloud(depth)
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(points)
    point_clouds.append(pc)

print(f"Generated {len(point_clouds)} point clouds.")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hope.KingsMan\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Hope.KingsMan\AppData\Local\Temp\ipykernel_14176\3755869466.py", line 26, in <module>
    points = depth_to_point_cloud(depth)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hope.KingsMan\AppData\Local\Temp\ipykernel_14176\3755869466.py", line None, in depth_to_point_cloud
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Hope.KingsMan\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Hope.KingsMan\AppData\Local\Programs\Python\Python311\Lib\site-packa

In [ ]:
# ICP registration
trajectory = []  # To store camera trajectory
for i in range(len(point_clouds) - 1):
    source = point_clouds[i]
    target = point_clouds[i + 1]
    
    # Apply ICP
    threshold = 0.02  # Distance threshold
    icp_result = o3d.pipelines.registration.registration_icp(
        source, target, threshold, 
        np.eye(4), 
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    
    # Save transformation matrix
    trajectory.append(icp_result.transformation)

    print(f"Transformation from frame {i} to {i + 1}:\n{icp_result.transformation}")


In [ ]:
import matplotlib.pyplot as plt

# Extract camera positions from transformations
camera_positions = [np.array([0, 0, 0])]
current_position = np.eye(4)

for transformation in trajectory:
    current_position = current_position @ transformation
    camera_positions.append(current_position[:3, 3])  # Extract translation

camera_positions = np.array(camera_positions)

# Plot the trajectory
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(camera_positions[:, 0], camera_positions[:, 1], camera_positions[:, 2], '-o')
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.title("Camera Trajectory")
plt.show()


In [ ]:
# Save point clouds as images
output_folder = "point_cloud_frames"
os.makedirs(output_folder, exist_ok=True)

for i, pc in enumerate(point_clouds):
    o3d.visualization.draw_geometries_with_animation_callback(
        [pc],
        lambda vis: vis.capture_screen_image(f"{output_folder}/frame_{i:04d}.png"),
        steps_per_frame=5,
    )

# Combine images into video
import cv2
import glob

images = sorted(glob.glob(f"{output_folder}/*.png"))
frame = cv2.imread(images[0])
height, width, _ = frame.shape
video = cv2.VideoWriter("output_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

for image_path in images:
    frame = cv2.imread(image_path)
    video.write(frame)

video.release()
print("Video created: output_video.mp4")
